# FBN Scraping 

## Creating Folder to Save Materials

In [1]:
import os

# Specify the folder name
folder_name = "19.2"

# Create the folder
os.makedirs(folder_name, exist_ok=True)

# Check if the folder was created
if os.path.exists(folder_name):
    print(f"Folder '{folder_name}' created successfully.")
else:
    print(f"Failed to create folder '{folder_name}'.")

Folder '19.2' created successfully.


## Converting From PDF to Text

In [2]:
#Importing the needed library to convert PDF to text
import pdfplumber

#extracting the PDF
def extract_text_from_pdf(pdf_path):
    with pdfplumber.open(pdf_path) as pdf:
        text = ''
        for page in pdf.pages:
            text += page.extract_text()
    return text

# Replace 'your_pdf_file.pdf' with the actual path to your PDF file
pdf_path = r"C:\Users\salom\OneDrive\Área de Trabalho\FBN New\FBN\Issues\19.2.pdf"

#variable containing text called extracted_text
extracted_text = extract_text_from_pdf(pdf_path)


In [3]:
#Downloading the text file 

with open('19.2/19.2_Issue_Text.txt', 'w', encoding='utf-8') as file:
    file.write(extracted_text)

## ISBNS

### Parsing ISBNs

In [4]:
#extracting ISBNs from thext 
#importing the library
import re

#defining the regex used 
def extract_isbns(text):
    # Define a regex pattern to match strings of numbers with dashes
    isbn_pattern = r"((978[-– ])?[0-9][0-9-– ]{10}[-– ][0-9xX])|((978)?[0-9]{9}[0-9Xx])"

    # Find all matches of the ISBN pattern in the text
    isbns = re.findall(isbn_pattern, text)

    return isbns

# Extract ISBNs from the example text
isbns_found = extract_isbns(extracted_text)

# Extract only the portions with numbers and dashes
isbns_found = [isbn[0] for isbn in isbns_found]

#print("Cleaned ISBNs:", isbns_found)


### ISBN Dataframe

In [5]:
#creating a dataframe that has ISBNs and the page numbers where found 

#importing necessary libaries
import pdfplumber
import pandas as pd
import re

def extract_isbns_from_text(text):
    # Define the ISBN regex pattern
    isbn_pattern = re.compile(r"((978[-– ])?[0-9][0-9-– ]{10}[-– ][0-9xX])|((978)?[0-9]{9}[0-9Xx])")

    # Find all ISBNs in the text
    isbns_found = isbn_pattern.findall(text)
    return isbns_found

def extract_text_from_pdf(pdf_path):
    data = {'PageNumber': [], 'Text': [], 'ISBNs': []}
    with pdfplumber.open(pdf_path) as pdf:
        for i, page in enumerate(pdf.pages, start=1):
            # Add page number and corresponding text to the data dictionary
            data['PageNumber'].append(i)
            text = page.extract_text()
            data['Text'].append(text)
            
            # Extract ISBNs from the text
            isbns_found = extract_isbns_from_text(text)
            isbns_found = [isbn[0] for isbn in isbns_found]
            data['ISBNs'].append(isbns_found)

    return pd.DataFrame(data)

# Replace 'your_pdf_file.pdf' with the actual path to your PDF file
pdf_path = r"C:\Users\salom\OneDrive\Área de Trabalho\FBN New\FBN\Issues\19.2.pdf"
df = extract_text_from_pdf(pdf_path)


In [6]:
df.head()

,PageNumber,Text,ISBNs
0,1,Feminist Bookstore News\nSource: Reveal Digita...,[]
1,2,Feminist\nBookstore\nNews fsi\nA\nS Jí\nSummer...,[]
2,3,と@ こ ジ の 3\noc\n3 h a nへk y oつu くpep レー 壮\n田\n...,[]
3,4,"Feminist Bookstore News\n0.00;0, 0 00-00 `C\nW...",[]
4,5,Looking for something\ngreat to read?\nu've co...,[]


### Downloading List of ISBNs and Page Numbers

In [7]:
#change issue number
df.to_csv('19.2/19.2_text.csv', index=False)

## API

In [ ]:
import requests
import csv
import time

api_key = '52084_9ddf62aeaa4a39b5485d9b7fb69dd5a8'
isbns_to_process = isbns_found  # Replace with the original list of ISBNs you want to process

# Removing Duplicates
isbns_to_process = list(set(isbns_to_process))

base_url = 'https://api2.isbndb.com/book/'

class RateLimiter:
    def __init__(self, calls_per_second):
        self.calls_per_second = calls_per_second
        self.interval = 1.0 / calls_per_second
        self.last_call = 0

    def wait(self):
        now = time.time()
        elapsed = now - self.last_call
        if elapsed < self.interval:
            time.sleep(self.interval - elapsed)
        self.last_call = time.time()

rate_limiter = RateLimiter(3)  # Allow 3 calls per second

with open('book_information.csv', 'w', newline='', encoding='utf-8') as csv_file:
    fieldnames = ['ISBN', 'Title', 'Author', 'Publisher', 'Pages', 'Date Published', 'Subjects', 'Binding', 'Synopsis', 'Language', 'Edition', 'Dimensions', 'MSRP', 'Image', 'Status']
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
    writer.writeheader()

    for isbn in isbns_to_process:
        url = f'{base_url}{isbn}'
        headers = {'Authorization': api_key}

        response = requests.get(url, headers=headers)

        if response.status_code == 200:
            book_info = response.json().get('book', {})
            writer.writerow({
                'ISBN': isbn,
                'Title': book_info.get('title', ''),
                'Author': ', '.join(book_info.get('authors', [])),
                'Publisher': book_info.get('publisher', ''),
                'Pages': book_info.get('pages', ''),
                'Date Published': book_info.get('date_published', ''),
                'Subjects': ', '.join(book_info.get('subjects', [])),  # Add subjects here
                'Binding': book_info.get('binding', ''),
                'Synopsis': book_info.get('synopsis', ''),
                'Language': book_info.get('language', ''),
                'Edition': book_info.get('edition', ''),
                'Dimensions': book_info.get('dimensions', ''),
                'MSRP': book_info.get('msrp', ''),
                'Image': book_info.get('image', ''),
                'Status': 'Success'
            })
        else:
            print(f"Error for ISBN {isbn}: {response.status_code}")
            writer.writerow({
                'ISBN': isbn,
                'Status': 'Error',
                'Title': '',
                'Author': '',
                'Publisher': '',
                'Pages': '',
                'Date Published': '',
                'Binding': '',
                'Synopsis': '',
                'Language': '',
                'Edition': '',
                'Dimensions': '',
                'MSRP': '',
                'Image': ''
            })

        rate_limiter.wait()  # Pause according to the rate limiter

print("CSV file created successfully.")

Error for ISBN : 403
Error for ISBN 0-931481-02-0: 404
Error for ISBN 139 0-88079-7: 404
Error for ISBN 4 0- 202 99 4: 404
Error for ISBN 24- 800-434-3: 404
Error for ISBN 1-56280-149-X: 404
Error for ISBN 31093256282 3: 404
Error for ISBN 312 404-627-2: 404
Error for ISBN 47- 612-641-0: 404
Error for ISBN 55406 1-800-2: 404
Error for ISBN 4386-61-328-1: 404
Error for ISBN 0-292-73120-0: 404
Error for ISBN 1-56280-136-8: 404
Error for ISBN 0 - 9 83 23 5: 404
Error for ISBN 0-9630360-5-x: 404
Error for ISBN 39 36 63 8- 2: 404
Error for ISBN 4 0 8- 474 -8: 404
Error for ISBN 0-674-21702-2: 404
Error for ISBN 4 0- 101 55 3: 404
Error for ISBN 9 1-612-729-0: 404
Error for ISBN 2 0-9648968-2: 404
Error for ISBN 0-935155-21-X: 404
Error for ISBN 1-56280-154-6: 404
Error for ISBN 20 928 09 --6: 404
Error for ISBN 0-89594-809-5: 404
Error for ISBN 4 0 8-7 44 -8: 404
Error for ISBN 0-935155-20-1: 404
Error for ISBN 10-527-0511-3: 404


## Joining Dataframes

### Renaming to ISBN

In [ ]:
import pandas as pd

# Assuming 'ISBNs' is the current column name and 'ISBN' is the desired column name
df.rename(columns={'ISBNs': 'ISBN'}, inplace=True)

### Adding Page Numbers

In [ ]:
page = df[['PageNumber', 'ISBN']]
page.head()

In [ ]:
page['ISBN'] = page['ISBN'].astype(str).str.replace('[\[\]\'\"]', '')
page.head()

In [ ]:
# Split values on comma and explode to create new rows
page['ISBN'] = page['ISBN'].str.split(', ')
page = page.explode('ISBN')

# Display the updated DataFrame
page.head()

### Opening Scraped Dataset

In [ ]:


# Specify the file path
file_path = r'C:\Users\salom\OneDrive\Área de Trabalho\FBN New\FBN\notebooks\book_information.csv'

# Read the CSV file into a DataFrame
info = pd.read_csv(file_path)

info.head()

### Joining it With Pages

In [ ]:
merge = pd.merge(info, page, on='ISBN', how='left')
merge

### Adding a Column For Issue and Volume 

In [ ]:
merge['Volume & Issue'] = '19.2'

### Removing Duplicates

In [ ]:
# Remove all duplicate rows from the DataFrame
merge = merge.drop_duplicates(keep=False)

# Reset the index after dropping duplicates
merge.reset_index(drop=True, inplace=True)

### Downloading Dataframe

In [ ]:
# Assuming 'merge' is your DataFrame
merge.to_csv('19.2/19.2_booklist.csv', index=False)